# PL10 - QSMVs con PennyLane

In [ ]:
!pip install pennylane

import pennylane as qml

## Creación del conjunto de datos

Utilizaremos las herramientas de *SciKit Learn* para crear un conjunto de datos con el que trabajar

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs

seed = 1234

X, y = make_blobs(n_samples=200, centers = [[0,2],[2,0]], random_state = seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=seed)
plt.scatter(X[:, 0], X[:, 1], c = y, cmap=matplotlib.colors.ListedColormap(["red","green"]));

Vamos a usar *angle embedding*, así que escalamos los datos al intervalo $[0,\pi]$.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

scaler = MinMaxScaler(feature_range=(0,np.pi))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## QUANTUM SUPPORT VECTOR MACHINES

Vamos a definir un *kernel* cuántico que calcularemos usando PennyLane

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def kernel(x1, x2):
    qml.templates.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.templates.AngleEmbedding(x2, wires=range(n_qubits)))
    return qml.probs()

def kernel_matrix(A, B):
    return np.array([[kernel(a, b)[0] for b in B] for a in A])

Ahora, usamos *SVC* de *SciKit Learn* para entrenar un SVM con el kernel que hemos construido

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)

# EJERCICIO

Crear un dataset de cuatro variables. Clasificarlo usando *QSVM*